In [24]:
from wandb.integration.keras import WandbCallback
import tensorflow as tf
import tensorflow_datasets as tfds
import wandb

In [25]:
wandb.login()
wandb.init(project="test", entity="hectordavid55555211", mode="offline")

In [26]:
(train, val, test), data_info = tfds.load("malaria",with_info=True,
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        shuffle_files=True, as_supervised=True)

In [27]:

def resize_only(image, label):
    # Redimensionar
    image = tf.image.resize(image, (224, 224))
    # Normalizar (opcional pero recomendado)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (train
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

val_dataset = (val
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

test_dataset = (test
                .map(resize_only, num_parallel_calls=AUTOTUNE)
                .batch(32)
                .prefetch(AUTOTUNE))

In [28]:
#inicializacion de wandb
config = wandb.config
config.learning_rate = 1e-3
config.batch_size = 32
config.epochs = 20

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    tf.keras.layers.Conv2D(16,3,activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(config.learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [30]:

train_batch = next(iter(train_dataset))

callbacks = [
    WandbCallback(
        save_model=False,
        log_weights=True,
        log_gradients=False,
        log_evaluation=True,
        training_data=train_batch,
    ),
    tf.keras.callbacks.ModelCheckpoint("ckpt_best.h5", save_best_only=True, monitor="val_loss")
]


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


In [31]:

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=config.epochs,
                    callbacks=callbacks)

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/20


689/689 [==============================] - 70s 100ms/step - loss: 0.5526 - accuracy: 0.7755 - val_loss: 0.4196 - val_accuracy: 0.8545
Epoch 2/20
Epoch 2/20


c:\Users\vpn\Documents\GitHub\Ciencia-de-datos\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


689/689 [==============================] - 67s 97ms/step - loss: 0.3774 - accuracy: 0.8709 - val_loss: 0.3942 - val_accuracy: 0.8556
Epoch 3/20
Epoch 3/20
689/689 [==============================] - 66s 95ms/step - loss: 0.3076 - accuracy: 0.8998 - val_loss: 0.3967 - val_accuracy: 0.8578
Epoch 4/20
Epoch 4/20
689/689 [==============================] - 65s 94ms/step - loss: 0.2665 - accuracy: 0.9170 - val_loss: 0.4334 - val_accuracy: 0.8356
Epoch 5/20
Epoch 5/20
689/689 [==============================] - 67s 97ms/step - loss: 0.2207 - accuracy: 0.9332 - val_loss: 0.4611 - val_accuracy: 0.8215
Epoch 6/20
Epoch 6/20
689/689 [==============================] - 66s 95ms/step - loss: 0.1890 - accuracy: 0.9476 - val_loss: 0.5301 - val_accuracy: 0.7932
Epoch 7/20
Epoch 7/20
689/689 [==============================] - 68s 99ms/step - loss: 0.1636 - accuracy: 0.9560 - val_loss: 0.6038 - val_accuracy: 0.7700
Epoch 8/20
Epoch 8/20
689/689 [==============================] - 69s 100ms/step - loss: 0.13

In [32]:
model.save("model_final.h5")
artifact = wandb.Artifact("malaria-cnn-model", type="model")
artifact.add_file("model_final.h5")
artifact.add_file("ckpt_best.h5")
wandb.log_artifact(artifact)
wandb.finish()

accuracy,▁▄▅▆▆▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
val_accuracy,███▇▆▅▃▃▂▃▁▁▁▃▁▂▃▃▁▁
val_loss,▁▁▁▁▁▂▂▃▃▄▄▅▅▅▆▇▆▇██
accuracy,0.99233
best_epoch,1
best_val_loss,0.39416
epoch,19
loss,0.03512
val_accuracy,0.73186


In [33]:
sweep_config = {
    "method": "bayes",   # "grid", "random", "bayes"
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "min": 1e-5,
            "max": 1e-2
        },
        "batch_size": {
            "values": [16, 32, 64]
        },
        "filters": {
            "values": [16, 32, 64]
        },
        "epochs": {
            "value": 10
        }
    }
}


In [34]:
sweep_id = wandb.sweep(sweep_config, project="test")
wandb.agent(sweep_id, function=train, count=20)

CommError: Expecting value: line 2 column 1 (char 1)